In [1]:
import os
os.getcwd()

'D:\\Kamie\\mon_ML'

In [2]:
import pandas as pd
import numpy as np

In [69]:
path = "../dataset/학습데이터처리_Data/auto-mpg.csv"

In [70]:
df = pd.read_csv(path, header=None)
df.columns = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model year', 'origin', 'name']

In [71]:
# 모든 열 출력하기
pd.set_option('display.max_column', None)

# 1. 데이터 표준화
: 익숙한 단위로 사용하기
- mpg -> kpl (km per liter)
- 1 mile = 1.690934 km
- 1 galon = 3.78541 L 

mpg to kpl = 1.690934 / 3.78541  # or about 0.425

In [72]:
mpg_to_kpl = 1.690934 / 3.78541

In [73]:
df['kpl'] = df['mpg'] * mpg_to_kpl

In [74]:
df.head(3)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name,kpl
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu,8.040559
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320,6.700466
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite,8.040559


# 2. 자료형 변환
컬럼의 속성을 분석 후 자료형 변환(변경)

In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    398 non-null    object 
 4   weight        398 non-null    float64
 5   acceleration  398 non-null    float64
 6   model year    398 non-null    int64  
 7   origin        398 non-null    int64  
 8   name          398 non-null    object 
 9   kpl           398 non-null    float64
dtypes: float64(5), int64(3), object(2)
memory usage: 31.2+ KB


=> horsepower은 숫자인데 object로 되어 있음 (물음표 포함되어 있어서 그럼)  

In [76]:
# horsepower의 "?"는 dataframe 변환 시 object로 인식하게 함
# -> np.nan으로 바꾼 뒤 지워주기
df["horsepower"].replace("?", np.nan, inplace=True)
df.dropna(subset=["horsepower"], axis=0, inplace=True)

In [77]:
# object -> float
df["horsepower"] = df["horsepower"].astype("float")
df["horsepower"].dtype  # 형변환 확인

dtype('float64')

# 3. 범주형 데이터 처리
1) numpy의 histogram을 이용해서 binning
2) pandas cut() 함수 이용해 범주형 데이터로 변환

In [78]:
# horsepower를 (low, norm, high)로 변경
count, bin_divider = np.histogram(df["horsepower"], bins=3)  # 3 분할
count, bin_divider  
# 숫자 4개 : |----|---|---| 이렇게 나누면 나눠지는 포인트 |가 4개라서.

(array([257, 103,  32], dtype=int64),
 array([ 46.        , 107.33333333, 168.66666667, 230.        ]))

### 범주형 데이터 변환을 위해 cut() 이용 (bin 이름 지정)

In [79]:
# bin 이름은 리스트로 제공.
bin_name = ['low', 'norm', 'high']

In [80]:
df['horsepower_bin'] = pd.cut(x=df["horsepower"],   # 적용할 데이터 배열(리스트)
                             bins=bin_divider,  # 범위 지정
                             labels=bin_name,  # 범위별 이름 지정
                             include_lowest=True  # 이상 or 초과 중 이상
                             )

In [81]:
df[["horsepower", "horsepower_bin"]].head(15)

,horsepower,horsepower_bin
0,130.0,norm
1,165.0,norm
2,150.0,norm
3,150.0,norm
4,140.0,norm
5,198.0,high
6,220.0,high
7,215.0,high
8,225.0,high
9,190.0,high


### low, norm, high -> [0,1,2] or 원핫인코딩으로 변경
pandas에서 one hot encoding : ```get_dummies()```

In [82]:
hp_dummies = pd.get_dummies(df["horsepower_bin"])
hp_dummies.head(15)

,low,norm,high
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
5,0,0,1
6,0,0,1
7,0,0,1
8,0,0,1
9,0,0,1


# 4. 정규화
: 각 컬럼의 데이터들을 해당 컬럼의 최댓값으로 나누는 방법  
$$ column \div column.max() $$
- 컬럼별 데이터의 범위가 차이가 많이 나면 학습에 영향을 미칠 수 있으므로 정규화 해주기
- 정규화하기 전 필요에 따라 이상치 제거해주면 좋음  
: 편차를 줄이기 위해 정규화를 하는 건데 이상치 제거 안하면 한 쪽으로 치우칠 수 있기 때문

- 그 데이터의 max값을 무조건 사용하는 게 아님.
    - ex) 기계 압력값을 정규화하는데 아직 가진 데이터 중에 최대 압력값이 없는 경우는 가진 데이터 중의 max값이 아니라 기계의 최대 압력값으로 정규화해줘야 함. 나중에 최댓값이 들어올 수 있기 때문!

### low, norm, high 상관없이 전체 정규화해주기

In [83]:
hs_max = df["horsepower"].max()
# df["horsepower"] = df["horsepower"] / hs_max / 2 - 1

### low, norm, high 별로 정규화해주기

In [68]:
df["horsepower"].max(), df["horsepower"].min()

(-0.5, -0.9)

In [134]:
# -1~1 사이로 변경해보기
values = [-6, -2, 2, 1, 3, 4]
max_l = max(list(map(abs, l)))   # == max([abs(val) for val in values]
normalize_data = [data / max_l for data in values]
normalize_data

[-1.0,
 -0.3333333333333333,
 0.3333333333333333,
 0.16666666666666666,
 0.5,
 0.6666666666666666]